In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score as auc
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from scipy.sparse import lil_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from scipy.sparse import hstack
import time


%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
train_df = pd.read_csv('train_sessions.csv')

In [3]:
train_df.shape[0]

253561

In [4]:
train_df.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
0,1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,3846.0,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
2,3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,39.0,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
3,4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,782.0,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
4,5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,178.0,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0


In [5]:
test_df = pd.read_csv('test_sessions.csv')

In [6]:
test_df.shape[0]

82797

In [7]:
test_df.head()

,session_id,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,site6,time6,site7,time7,site8,time8,site9,time9,site10,time10
0,1,29,2014-10-04 11:19:53,35.0,2014-10-04 11:19:53,22.0,2014-10-04 11:19:54,321.0,2014-10-04 11:19:54,23.0,2014-10-04 11:19:54,2211.0,2014-10-04 11:19:54,6730.0,2014-10-04 11:19:54,21.0,2014-10-04 11:19:54,44582.0,2014-10-04 11:20:00,15336.0,2014-10-04 11:20:00
1,2,782,2014-07-03 11:00:28,782.0,2014-07-03 11:00:53,782.0,2014-07-03 11:00:58,782.0,2014-07-03 11:01:06,782.0,2014-07-03 11:01:09,782.0,2014-07-03 11:01:10,782.0,2014-07-03 11:01:23,782.0,2014-07-03 11:01:29,782.0,2014-07-03 11:01:30,782.0,2014-07-03 11:01:53
2,3,55,2014-12-05 15:55:12,55.0,2014-12-05 15:55:13,55.0,2014-12-05 15:55:14,55.0,2014-12-05 15:56:15,55.0,2014-12-05 15:56:16,55.0,2014-12-05 15:56:17,55.0,2014-12-05 15:56:18,55.0,2014-12-05 15:56:19,1445.0,2014-12-05 15:56:33,1445.0,2014-12-05 15:56:36
3,4,1023,2014-11-04 10:03:19,1022.0,2014-11-04 10:03:19,50.0,2014-11-04 10:03:20,222.0,2014-11-04 10:03:21,202.0,2014-11-04 10:03:21,3374.0,2014-11-04 10:03:22,50.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:22,48.0,2014-11-04 10:03:23,3374.0,2014-11-04 10:03:23
4,5,301,2014-05-16 15:05:31,301.0,2014-05-16 15:05:32,301.0,2014-05-16 15:05:33,66.0,2014-05-16 15:05:39,67.0,2014-05-16 15:05:40,69.0,2014-05-16 15:05:40,70.0,2014-05-16 15:05:40,68.0,2014-05-16 15:05:40,71.0,2014-05-16 15:05:40,167.0,2014-05-16 15:05:44


In [8]:
#выделим колонки с сайтами

site_columns = ['site' + str(i) for i in range(1,11)]

In [9]:
#выделим колонки с временными фичами

time_columns = ['time' + str(i) for i in range(1,11)]

In [10]:
#объединим трейн и тест в один датасет

train_test_df = pd.concat([train_df, test_df], sort = False)

In [11]:
train_test_df.shape[0]

336358

In [12]:
#сначала создадим спарс-матрицу для колонок с сайтами

train_test_df[site_columns] = train_test_df[site_columns].fillna(-1)

In [13]:
train_test_df[site_columns] = train_test_df[site_columns].astype(int)

In [14]:
def find_sites(li):
    lli = {}
    for l in li:
        if l > 0:
            if l in lli:
                lli[l] += 1
            else:
                lli[l] = 1
    return lli

In [15]:
ttd = train_test_df[site_columns].copy()

In [16]:
ttd['all_sites'] = ttd.apply(find_sites, axis = 1)

In [17]:
ttd.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,all_sites
0,718,-1,-1,-1,-1,-1,-1,-1,-1,-1,{718: 1}
1,890,941,3847,941,942,3846,3847,3846,1516,1518,"{890: 1, 941: 2, 3847: 2, 942: 1, 3846: 2, 151..."
2,14769,39,14768,14769,37,39,14768,14768,14768,14768,"{14769: 2, 39: 2, 14768: 5, 37: 1}"
3,782,782,782,782,782,782,782,782,782,782,{782: 10}
4,22,177,175,178,177,178,175,177,177,178,"{22: 1, 177: 4, 175: 2, 178: 3}"


In [18]:
ttd['len_sites'] = ttd['all_sites'].apply(lambda x: sum(x.values()))

In [19]:
ttd.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10,all_sites,len_sites
0,718,-1,-1,-1,-1,-1,-1,-1,-1,-1,{718: 1},1
1,890,941,3847,941,942,3846,3847,3846,1516,1518,"{890: 1, 941: 2, 3847: 2, 942: 1, 3846: 2, 151...",10
2,14769,39,14768,14769,37,39,14768,14768,14768,14768,"{14769: 2, 39: 2, 14768: 5, 37: 1}",10
3,782,782,782,782,782,782,782,782,782,782,{782: 10},10
4,22,177,175,178,177,178,175,177,177,178,"{22: 1, 177: 4, 175: 2, 178: 3}",10


In [20]:
sp_list = list(ttd['all_sites'])

In [21]:
ttd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336358 entries, 0 to 82796
Data columns (total 12 columns):
site1        336358 non-null int32
site2        336358 non-null int32
site3        336358 non-null int32
site4        336358 non-null int32
site5        336358 non-null int32
site6        336358 non-null int32
site7        336358 non-null int32
site8        336358 non-null int32
site9        336358 non-null int32
site10       336358 non-null int32
all_sites    336358 non-null object
len_sites    336358 non-null int64
dtypes: int32(10), int64(1), object(1)
memory usage: 20.5+ MB


In [22]:
sp_list

[{718: 1},
 {890: 1, 941: 2, 3847: 2, 942: 1, 3846: 2, 1516: 1, 1518: 1},
 {14769: 2, 39: 2, 14768: 5, 37: 1},
 {782: 10},
 {22: 1, 177: 4, 175: 2, 178: 3},
 {570: 2, 21: 3},
 {803: 2, 23: 1, 5956: 1, 17513: 1, 37: 1, 21: 1, 17514: 3},
 {22: 1, 21: 2, 29: 1, 5041: 2, 14422: 1, 23: 1, 14421: 2},
 {668: 1, 940: 2, 942: 2, 941: 2, 23: 1, 21: 1, 22: 1},
 {3700: 1, 229: 2, 570: 1, 21: 4, 2336: 1, 2044: 1},
 {229: 1, 1500: 2, 33: 1, 391: 1, 35: 1, 29: 1, 2276: 1, 40305: 1, 23: 1},
 {37: 1, 39: 1, 3592: 2, 890: 1, 35: 1, 29: 1, 22: 1, 30: 1, 33: 1},
 {23: 3, 747: 1, 749: 1, 21: 1, 752: 1, 55: 2, 56: 1},
 {262: 1, 322: 1, 625: 1, 52: 1, 37: 1, 152: 1, 21: 1, 2164: 2, 30: 1},
 {2883: 1, 2945: 1, 167: 3, 21: 2, 3: 1, 229: 1, 76: 1},
 {28395: 1,
  248: 1,
  728: 1,
  3364: 1,
  3362: 1,
  1434: 1,
  30: 1,
  102: 1,
  727: 1,
  28396: 1},
 {63: 1, 167: 5, 359: 1, 360: 1, 363: 1, 362: 1},
 {178: 1, 343: 1, 167: 2, 270: 1, 2726: 1, 2727: 1, 2725: 1, 2724: 1, 2732: 1},
 {22736: 10},
 {33: 1, 35: 1, 

In [23]:
site1 = lil_matrix((ttd.shape[0], 100000))
row = 0

for s in sp_list:
    for key, value in s.items():
        site1[row, key] = 1
    row += 1

site1 = site1.tocsc()[:, np.where(site1.getnnz(axis = 0) > 0)[0]].tocsr()


In [24]:
ttest = site1[train_df.shape[0]:]

In [25]:
site1 = site1.tocsc()[:, np.where((ttest.getnnz(axis=0) > 0))[0]].tocsr()

In [26]:
ttest = site1[train_df.shape[0]:]
site1 = site1[:train_df.shape[0]]

In [27]:
site1

<253561x15857 sparse matrix of type '<class 'numpy.float64'>'
	with 1320910 stored elements in Compressed Sparse Row format>

In [28]:
ttest

<82797x15857 sparse matrix of type '<class 'numpy.float64'>'
	with 437222 stored elements in Compressed Sparse Row format>

In [29]:
y_train = train_df.target

In [30]:
#теперь разберемся с временными колонками

In [31]:
train_test_df[time_columns] = train_test_df[time_columns].apply(pd.to_datetime)

In [32]:
#создадим отдельный дадафрейм, куда будем записывать фичи, которые будут выведены
#из исходных временных фич

time_df = pd.DataFrame(index = train_test_df.index) 

In [33]:
#продолжительность сессии в минутах

time_df['minutes'] = round(((train_test_df[time_columns].max(axis = 1) - train_test_df[time_columns].min(axis = 1)) / np.timedelta64(1, 'm')), 2)

In [34]:
#день недели (day of week)
#посмотрим, в какие дни недели была активность,
#выделим в днях выходные и будни, а также рабочие часы

time_df['dow'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: ts.date().weekday())
time_df['is_weekend'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: 1 if ts.date().weekday() in (5, 6) else 0)
time_df['work_hours'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: 1 if (ts.date().weekday() in (0, 1, 2, 3)) &
                                             ((ts.hour >= 8) & (ts.hour <= 17) & (ts.hour != 12)) else 0)


In [35]:
#неделя, когда началась сессия
#день, когда началась сессия
#час, когда началась сессия
#год+месяц начала

time_df['start_week'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: ts.week)
time_df['start_day'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: ts.timetuple().tm_yday)
time_df['start_hour'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: ts.hour)
time_df['start_month'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: 100 * ts.year + ts.month)

In [36]:
#год
#месяц

time_df['year'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: ts.year)
time_df['month'] = train_test_df[time_columns].min(axis = 1).apply(lambda ts: ts.month)

In [37]:
time_df.head()

,minutes,dow,is_weekend,work_hours,start_week,start_day,start_hour,start_month,year,month
0,0.00,3,0,1,8,51,10,201402,2014,2
1,0.43,5,1,0,8,53,11,201402,2014,2
2,0.12,0,0,1,51,350,16,201312,2013,12
3,4.50,4,0,0,13,87,10,201403,2014,3
4,4.10,4,0,0,9,59,10,201402,2014,2


In [38]:
#сделаем one-hot encoding всех временных фич

X_dummied = pd.get_dummies(time_df, columns = time_df.columns, drop_first = True)

In [39]:
#теперь объединим спарс-матрицу сайтов и датасет с временными значениями для трейна и теста

In [40]:
X_train = hstack([site1, X_dummied[:train_df.shape[0]]])

In [41]:
X_test = hstack([ttest, X_dummied[train_df.shape[0]:]])

In [42]:
logr = LogisticRegression(random_state = 42, C = 1.4)

In [43]:
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

In [44]:
#посмотрим roc_auc_score на кросс-валидации

score = cross_val_score(estimator = logr, X = X_train, y = y_train, cv = skf, scoring = 'roc_auc').mean()
print('roc_auc_score: ', round(score, 4))

roc_auc_score:  0.9934


In [45]:
#посмотрим accuracy на кросс-валидации

score2 = cross_val_score(estimator = logr, X = X_train, y = y_train, cv = skf, scoring = 'accuracy').mean()
print('accuracy: ', round(score2, 4))

accuracy:  0.9955


In [46]:
#функция для записи предсказаний в файл

def write_to_submission_file(predicted_labels, out_file,
                             target = 'target', index_label = "session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns = [target])
    predicted_df.to_csv(out_file, index_label = index_label)

In [47]:
logr.fit(X_train, y_train)
y_pred = logr.predict_proba(X_test)

In [48]:
y_pred[:, 1]

array([8.23513004e-06, 1.75327360e-06, 5.45426527e-06, ...,
       8.52943838e-06, 7.18280234e-08, 4.20102081e-07])

In [49]:
write_to_submission_file(y_pred[:, 1], 'submission_Alice.csv')

In [50]:
#мой скор на кегле 0.95696